In [1]:
import json
import prompt_rag
import torch
import os
import sys

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../..", ".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import covmis, liar2

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig
)
from custom import CustomModelType

# model_type = CustomModelType.mixtral_moe_7b_instruct_awq
model_type = CustomModelType.llama_3_70b_instruct_awq
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    gpu_memory_utilization=0.92,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    max_num_seqs=64,
    engine_kwargs = {
        # "enforce_eager": True,
        "seed": 42,
    }
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_tokens=2048,
    temperature=0,
    seed=42
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True, 
)



[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] No LMDeploy installed, if you are using LMDeploy, you will get `ImportError: cannot import name 'prepare_lmdeploy_engine_template' from 'swift.llm'`
[INFO:swift] Loading the model using model_dir: /home/css/models/llama-3-70b-instruct-awq
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/llama-3-70b-instruct-awq",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 8192,
  "initializer_range": 0.02,
  "intermediate_size": 28672,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 64,
  "num_hidden_layers": 80,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "bits": 4,
    

INFO 10-12 17:19:19 awq_marlin.py:90] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 10-12 17:19:19 config.py:899] Defaulting to use mp for distributed inference
INFO 10-12 17:19:19 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='/home/css/models/llama-3-70b-instruct-awq', speculative_config=None, tokenizer='/home/css/models/llama-3-70b-instruct-awq', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=Observabili

Loading safetensors checkpoint shards:   0% Completed | 0/9 [00:00<?, ?it/s]


(VllmWorkerProcess pid=428760) INFO 10-12 17:19:33 model_runner.py:1025] Loading model weights took 18.5514 GB
INFO 10-12 17:19:33 model_runner.py:1025] Loading model weights took 18.5514 GB
INFO 10-12 17:19:37 distributed_gpu_executor.py:57] # GPU blocks: 885, # CPU blocks: 1638
INFO 10-12 17:19:39 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=428760) INFO 10-12 17:19:39 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=428760) INFO 10-12 17:19:39 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilizati

In [2]:
K = 5
sort = False

prior_knowledge_version = "1"
search_engine = "brave"
model_name = 'llama3'
dataset = 'covmis' # liar2 covmis
wsc = True

data_type = 'entire'
search_date = None
# data_search = data_search[:10] + [data_search[9690]]
prompt_rag.update_train_search_llm(
    model_name, get_resp_list, search_engine,
    dataset, prior_knowledge_version,
    data_type=data_type, search_date=search_date,
    K=K, sort=sort, wsc=wsc
)


  0%|          | 0/14384 [00:00<?, ?it/s]

WARNING 10-12 17:20:43 scheduler.py:1439] Sequence group 9 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


  2%|▏         | 258/14384 [11:44<9:05:35,  2.32s/it] 

WARNING 10-12 17:32:21 scheduler.py:1439] Sequence group 265 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


  3%|▎         | 477/14384 [21:37<6:09:18,  1.59s/it] 

...

 98%|█████████▊| 14032/14384 [10:19:35<15:45,  2.69s/it]

WARNING 10-13 03:40:11 scheduler.py:1439] Sequence group 14040 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2901


 99%|█████████▉| 14302/14384 [10:31:27<02:45,  2.02s/it]

WARNING 10-13 03:52:02 scheduler.py:1439] Sequence group 14309 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=2951


100%|██████████| 14384/14384 [10:34:57<00:00,  2.65s/it]
